In [7]:
import torch
import pandas as pd
from src.process.stats_encoder import StatsEncoder

In [3]:
df = pd.read_csv("data/dataset.csv")
df.head()


,date,MP,PTS,FG%,TRB,AST,STL,BLK,TOV,PF,Result,question,answer
0,2007-05-21,45:22,10.0,0.333,10.0,9.0,4.0,1.0,2.0,1.0,L,"I know you say that you're a football player, ...",(Laughing) It was definitely a physical game t...
1,2007-05-21,45:22,10.0,0.333,10.0,9.0,4.0,1.0,2.0,1.0,L,"The last play there, Coach said that was kind ...","No, I go for the winning play. If two guys co..."
2,2007-05-21,45:22,10.0,0.333,10.0,9.0,4.0,1.0,2.0,1.0,L,"As a franchise player, how do you justify only...","No, you've just got to take what's there. It'..."
3,2007-05-21,45:22,10.0,0.333,10.0,9.0,4.0,1.0,2.0,1.0,L,Coach said that a couple of adjustments need t...,We definitely played pretty well. Both teams ...
4,2007-05-21,45:22,10.0,0.333,10.0,9.0,4.0,1.0,2.0,1.0,L,"Third quarter, again, you guys fell behind 7-0...",Not sure. It's just something that we've got ...


In [ ]:
numeric_columns = ['PTS', 'AST', 'TRB', 'STL', 'BLK', 'FG%', 'TOV', 'PF', 'Result']
stats_data = df[numeric_columns]

stats_tensor = torch.tensor(stats_data.values, dtype=torch.float32)

print(stats_tensor.shape)


torch.Size([1193, 8])


In [5]:
input_dim = stats_tensor.shape[1] 
encoder = StatsEncoder(input_dim=input_dim)


In [6]:
output = encoder(stats_tensor)

print("Input shape:", stats_tensor.shape)
print("Output shape:", output.shape)
print("Sample output embedding:", output[0])


Input shape: torch.Size([1193, 8])
Output shape: torch.Size([1193, 32])
Sample output embedding: tensor([ 0.5795, -0.3090, -0.2761, -0.6459, -0.9088,  0.8734,  0.3222,  0.7145,
        -0.3681,  0.3558, -0.2537,  0.1247, -0.1205,  0.6015, -0.0999,  0.0406,
        -0.9596, -0.1728,  0.0920, -0.1629,  0.7647, -0.1422, -1.2084,  0.9715,
         0.2264,  0.5450, -0.4997, -0.6741, -0.5831,  0.9650,  1.1721,  0.1893],
       grad_fn=<SelectBackward0>)
